In [1]:
#|default_exp testing
#|export

from LendingClubAutoencoder import preprocessing, autoencoders, training

import torch

from datetime import datetime, timedelta

In [2]:
#|test

import json

In [1]:
#|export

def test_vae(model_file_name, test_loader, sigmoid_mask, binary_mask, device):
    
    model = training.get_best_model(autoencoders.VariationalAutoencoder, sigmoid_mask, model_file_name)
    
    model.eval()
    test_total_loss = 0
    test_rmse_loss = 0
    test_kl_loss = 0
    n_batches = 0

    with torch.no_grad():
        for data_batch, mask_batch in test_loader:
            data_batch, mask_batch = data_batch.to(device), mask_batch.to(device)
            reconstruction, mean = model(data_batch)
            
            total_loss, rmse_loss, kl_loss = training.masked_vae_loss(reconstruction, data_batch, mask_batch, binary_mask)
            
            test_total_loss = test_total_loss + total_loss.item()
            test_rmse_loss = test_rmse_loss + rmse_loss.item()
            test_kl_loss = test_kl_loss + kl_loss.item()
            n_batches = n_batches + 1

    average_total_loss = test_total_loss / n_batches
    average_rmse_loss = test_rmse_loss / n_batches
    average_kl_loss = test_kl_loss / n_batches

    return average_total_loss, average_rmse_loss, average_kl_loss



In [8]:
#|export

def cross_validate_vae(lending_club_data_handler: preprocessing.DataHandler, start: int, end: int, learning_rate: float = 1e-4, n_folds: int = 5, ):
    '''
    Perform k-fold cross-validation for the VAE
    
    Args:
        start (datetime): Start date
        end (datetime): End date
        n_folds (int): Number of folds
    '''

    total_days = (end - start).days

    train_size = 0.7
    validation_size = 0.15
    test_size = 0.15

    window_size = total_days / (1+train_size*(n_folds-1))
    train_step_size = window_size * train_size
    validation_step_size = window_size * validation_size
    test_step_size = window_size * test_size

    fold_test_losses = {}

    current_start = start

    for fold in range(n_folds):
        train_start = current_start
        train_end = train_start + timedelta(days=int(train_step_size))

        validation_start = train_end + timedelta(days=1)
        validation_end = validation_start + timedelta(days=int(validation_step_size))

        test_start = validation_end + timedelta(days=1)
        test_end = test_start + timedelta(days=int(test_step_size))

        current_start = validation_start
        
        # Ensure we don't go past the end date
        if test_end > end:
            test_end = end

        train_data, train_mask = lending_club_data_handler.get_train_data(train_start, train_end)
        validation_data, validation_mask = lending_club_data_handler.get_test_data(validation_start, validation_end)
        test_data, test_mask = lending_club_data_handler.get_test_data(test_start, test_end)

        train_loader = preprocessing.to_torch_dataloader(train_data,train_mask)
        validation_loader = preprocessing.to_torch_dataloader(validation_data,validation_mask)
        test_loader = preprocessing.to_torch_dataloader(test_data,test_mask)


        # Instantiate model and optimiser 
        model = autoencoders.VariationalAutoencoder(input_size=len(train_data[0]))
        optimiser = torch.optim.Adam(model.parameters(), lr=learning_rate)#original is 1e-3

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # Train model for this fold 
        training.train_variational_autoencoder(model, optimiser, train_loader, validation_loader, device=device)

        # Evaluate best model for this fold
        model_file_name = f'trained_models/vae_best-input_size:{len(train_data[0])}.pt'
        
        test_total_loss, test_mse_loss, test_kl_loss = test_vae(model_file_name, test_loader, device)

        fold_test_losses[fold]={'total_loss':test_total_loss, 'mse_loss':test_mse_loss, 'kl_loss':test_kl_loss}
        print(f'Test Loss for Fold {fold+1}: {test_total_loss:.4f}')

    return fold_test_losses

In [ ]:
#|test

start = datetime(2007, 1, 1)
end = datetime(2017, 5, 31)

results = {}
results['cv_results'] = cross_validate_vae(start, end)
json.dump(results, open('cv_results.json', 'w'))

In [ ]:
#|test

lending_club_data_handler = preprocessing.DataHandler(csv_path='../local_data/all_lending_club_loan_data_2007-2018.csv')

train_start = datetime(2007, 1, 1)
train_end = datetime(2017, 5, 31)

validation_start = datetime(2017, 6, 1)
validation_end = datetime(2017, 12, 31)


train_data, train_mask = lending_club_data_handler.get_train_data(train_start, train_end)
validation_data, validation_mask = lending_club_data_handler.get_test_data(validation_start, validation_end)

train_loader = preprocessing.to_torch_dataloader(train_data,train_mask)
validation_loader = preprocessing.to_torch_dataloader(validation_data,validation_mask)


# Instantiate model and optimiser 
model = autoencoders.VariationalAutoencoder(input_size=len(train_data[0]))
optimiser = torch.optim.Adam(model.parameters(), lr=1e-3)#original is 1e-3

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train model for this fold 
training.train_variational_autoencoder(model, optimiser, train_loader, validation_loader, device=device)



In [77]:
#|test

# Evaluate best model for this fold
test_start = datetime(2018, 1, 1)
test_end = datetime(2018, 12, 31)

test_data, test_mask = lending_club_data_handler.get_test_data(test_start, test_end)
test_loader = preprocessing.to_torch_dataloader(test_data,test_mask)

model_file_name = f'../trained_models/vae_best-input_size:{len(train_data[0])}.pt'

total_loss, rmse_loss, kl_loss = test_vae(model_file_name, test_loader, device)

In [ ]:
for start_month, end_month in zip([1,4,7,10], [3,6,9,12]):
    start_date = datetime(2018, start_month, 1)
    
    try:
        end_date = datetime(2018, end_month, 31)
    except ValueError:
        end_date = datetime(2018, end_month, 30)

    test_data, test_mask = lending_club_data_handler.get_test_data(test_start, test_end)
    test_loader = preprocessing.to_torch(test_data,test_mask)

    model_file_name = f'../trained_models/vae_best-input_size:{len(train_data[0])}.pt'

    total_loss, rmse_loss, kl_loss = test_vae(model_file_name, test_loader, device)

    print(total_loss, rmse_loss, kl_loss)